In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import time
import pandas as pd
from datetime import datetime
from selenium.webdriver.common.by import By
import random

sleep_time = random.uniform(1, 2)


# 크롬 디버거로 크롬 구동
subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"')
time.sleep(3)  # 크롬이 완전히 실행될 때까지 대기

# 웹드라이버 설정
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
driver_path = f'./{chrome_ver}/chromedriver.exe'
option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

try:
    driver = webdriver.Chrome(driver_path, options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(driver_path, options=option)

driver.implicitly_wait(5)  # 웹 자원 로드를 위해 5초까지 기다림

input("홈페이지 접속하면 ENTER")

In [ ]:
import pandas as pd
from datetime import datetime

# CSV 파일을 읽어 DataFrame으로 로드합니다.
file_path = 'datasets.csv'
df = pd.read_csv(file_path)

# 'Date' 칼럼을 datetime 타입으로 변환합니다.
df['Date'] = pd.to_datetime(df['Date'])

# 사용자로부터 시작 날짜와 종료 날짜를 입력받습니다.
start_date_input = input("시작 날짜를 입력하세요 (YYYY-MM-DD): ")
end_date_input = input("종료 날짜를 입력하세요 (YYYY-MM-DD): ")

# 입력받은 날짜 문자열을 datetime 객체로 변환합니다.
start_date = datetime.strptime(start_date_input, '%Y-%m-%d')
end_date = datetime.strptime(end_date_input, '%Y-%m-%d')

# 지정된 날짜 범위로 데이터를 필터링합니다.
filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

# 필터링된 DataFrame에서 'Date' 칼럼만 추출하여 리스트로 변환합니다.
formatted_date_list = filtered_df['Date'].dt.strftime('%Y%m%d').tolist()

# 결과 확인을 위해 상위 5개 날짜를 출력합니다.
print(formatted_date_list[:5])

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np  # NaN 처리를 위해 import
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time

# 결과를 저장할 DataFrame 생성. 초기에는 'Date' 칼럼만 있는 상태입니다.
result_df = pd.DataFrame(columns=['Date'])

# 찾고자 하는 문자열 리스트
target_strings = [
    '코스피 200', '코스피 200 건설', '코스피 200 중공업', '코스피 200 철강/소재',
    '코스피 200 에너지/화학', '코스피 200 정보기술',
    '코스피 200 금융', '코스피 200 생활소비재', '코스피 200 경기소비재'
]

# 각 문자열에 대해 칼럼을 추가합니다.
for idx, _ in enumerate(target_strings, start=1):
    result_df[f"{idx}_PER"] = None
    result_df[f"{idx}_PBR"] = None

# 오류 로그를 저장할 리스트
error_log = []

def click_search_button_with_dates(driver, formatted_date_list, input_xpath, search_button_xpath):
    for date_str in formatted_date_list:
        try:
            # 날짜 입력 필드 찾기
            date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, input_xpath)))
            # 날짜 입력 필드에 값 입력
            date_input.clear()
            date_input.send_keys(date_str)
            
            # 검색 버튼 클릭
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, search_button_xpath)))
            search_button.click()
            time.sleep(5)  # 페이지 로딩을 위해 잠시 대기
            
            # 데이터 추출 및 저장
            extract_and_save_data(driver, date_str)
            
            # CSV 파일로 저장하고 상태를 출력합니다.
            result_df.to_csv('2014_result.csv', index=False)
            print(f"{date_str} - Data has been successfully saved to interim_result.csv")
            
        except Exception as e:  # 모든 예외를 포착합니다.
            print(f"Error with date {date_str}: {e}")
            error_log.append(f"Error with date {date_str}: {e}")
            
            # CSV 파일로 저장하고 상태를 출력합니다.
            result_df.to_csv('interim_result.csv', index=False)
            print(f"{date_str} - Data has been successfully saved to interim_result.csv despite the error")

def extract_and_save_data(driver, date_str):
    # 현재 페이지의 소스를 가져옵니다.
    page_source = driver.page_source
    
    # BeautifulSoup 객체를 생성합니다.
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # 결과를 임시로 저장할 딕셔너리
    result_dict = {"Date": date_str}
    
    # 각 문자열에 대해 처리합니다.
    for idx, target_string in enumerate(target_strings, start=1):
        # 문자열을 포함하는 td 태그를 찾기
        target_td = soup.find('td', string=target_string)
        
        if target_td:
            # target_td의 형제들 중에서 data-bind="WT_PER"와 "WT_STKPRC_NETASST_RTO" 속성을 가진 태그의 텍스트를 추출
            per_tag = target_td.find_next_sibling(attrs={'data-bind': 'WT_PER'})
            pbr_tag = target_td.find_next_sibling(attrs={'data-bind': 'WT_STKPRC_NETASST_RTO'})
            
            if per_tag and pbr_tag:
                try:
                    PER = float(per_tag.get_text())
                except ValueError:  # 숫자로 변환할 수 없는 경우
                    PER = np.nan  # NaN으로 설정
                
                try:
                    PBR = float(pbr_tag.get_text())
                except ValueError:  # 숫자로 변환할 수 없는 경우
                    PBR = np.nan  # NaN으로 설정
                
                # 결과 딕셔너리에 값 추가
                result_dict[f"{idx}_PER"] = PER
                result_dict[f"{idx}_PBR"] = PBR
            else:
                print(f"{date_str} 날짜, {target_string}에 대한 필요한 태그를 찾지 못했습니다.")
        else:
            print(f"{date_str} 날짜에 {target_string} 문자열을 포함하는 태그를 찾지 못했습니다.")
    
    # 결과 DataFrame에 행을 추가합니다.
    result_df.loc[len(result_df)] = result_dict

# 함수 호출
input_xpath = '//input[@id="trdDd"]'
search_button_xpath = '//*[@id="jsSearchButton"]'
click_search_button_with_dates(driver, formatted_date_list, input_xpath, search_button_xpath)

# 결과 DataFrame 출력
print(result_df)

# 오류 로그 출력
print(error_log)

# 최종 결과를 CSV 파일로 저장
result_df.to_csv('2014_result.csv', index=False)